<h1 style="text-align: center; font-size: 50px;"> Register Model </h1>

# Notebook Overview

- Configure the Environment
- Define Constants and Paths
- Load Configuratons and Secrets
- Register Model 
- Evaluate Model
- Log Execution Time

In [ ]:
import logging
import time
from datetime import datetime
from pathlib import Path

# Configure logger
logger: logging.Logger = logging.getLogger("register_markdown_model_logger")
logger.setLevel(logging.INFO)
logger.propagate = False

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
if not logger.handlers:
    stream_handler: logging.StreamHandler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

In [ ]:
start_time = time.time()
logger.info("Notebook execution started.")

In [ ]:
# File Paths
CONFIG_PATH = Path("../configs/configs.yaml")
SECRETS_PATH = Path("../configs/secrets.yaml")
REQUIREMENTS_PATH = Path("../requirements.txt")
CODE_PATH = Path("../src")
LOCAL_MODEL_PATH = Path("/home/jovyan/datafabric/llama3.1-8b-instruct/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf")

# Evaluation Data Path
EVAL_DATA_PATH = Path("./results.json")

# MLflow Configuration
EXPERIMENT_NAME = "markdown-correction-experiment"
RUN_NAME = f"registration-run-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
MODEL_NAME = "MarkdownCorrector"

In [ ]:
%%time
# %pip install -r {REQUIREMENTS_PATH} --quiet

In [ ]:
# Standard & Third-Party Libraries
import json
import os
import sys
import pandas as pd
import mlflow
from mlflow.models import evaluate, ModelSignature
from mlflow.types import Schema, ColSpec
from mlflow.metrics import ari_grade_level, exact_match, rouge1, rougeL
from mlflow.tracking import MlflowClient

# ✅ Add src directory to system path. This allows the notebook to find
# your custom modules for interactive development.
sys.path.append(str(CODE_PATH.resolve()))

# Internal Modules (now imported correctly from the src directory)
from markdown_correction_service import MarkdownCorrectionService
from utils import load_config_and_secrets
from llm_metrics import (
    semantic_similarity_metric,
    grammar_error_count_metric,
    grammar_error_rate_metric,
    grammar_improvement_metric,
    grammar_score_metric,
    readability_improvement_metric,
    llm_judge_metric_local,
)

In [ ]:
def log_asset_status(asset_path: Path, asset_name: str) -> None:
    """Logs the status of a given asset based on its existence."""
    if asset_path.exists():
        logger.info(f"✅ {asset_name} is properly configured at: {asset_path}")
    else:
        logger.warning(f"⚠️ {asset_name} not found at: {asset_path}. Please ensure it is correctly configured.")

# --- Validate Assets ---
log_asset_status(CONFIG_PATH, "Config file")
log_asset_status(SECRETS_PATH, "Secrets file")
log_asset_status(LOCAL_MODEL_PATH, "Local LLaMA model")
log_asset_status(EVAL_DATA_PATH, "Evaluation data JSON")
log_asset_status(REQUIREMENTS_PATH, "Requirements file")

In [ ]:
# Load evaluation results from the JSON file
with open(EVAL_DATA_PATH, "r") as f:
    results = json.load(f)

# Extract original markdown texts and create evaluation DataFrame
original_texts = [item["original"] for item in results]
eval_df = pd.DataFrame(original_texts, columns=["markdown"])

logger.info(f"Loaded {len(eval_df)} records for evaluation.")
print(eval_df.head())

In [ ]:
# %%time
mlflow.set_experiment(EXPERIMENT_NAME)

with mlflow.start_run(run_name=RUN_NAME) as run:
    run_id = run.info.run_id
    logger.info(f"MLflow Run Started. Run ID: {run_id}")

    # Use the classmethod from your module to log and register the model
    MarkdownCorrectionService.log_model(
        model_name=MODEL_NAME,
        llm_artifact_path=str(LOCAL_MODEL_PATH),
        config_path=str(CONFIG_PATH),
        secrets_path=str(SECRETS_PATH),
        requirements_path=str(REQUIREMENTS_PATH),
        # ✅ This parameter is the key fix. It tells MLflow to bundle the
        # entire contents of the '../src' directory with the model artifact.
        code_paths=[str(CODE_PATH)]
    )
    
    model_uri = f"runs:/{run_id}/{MODEL_NAME}"
    logger.info(f"Model URI: {model_uri}")

In [ ]:
client = MlflowClient()
try:
    latest_version_info = client.get_latest_versions(MODEL_NAME, stages=["None"])[0]
    latest_version = latest_version_info.version
    logger.info(f"Successfully registered model '{MODEL_NAME}' version {latest_version}.")
    
    # Load model for a test prediction
    model_uri_latest = f"models:/{MODEL_NAME}/{latest_version}"
    loaded_model = mlflow.pyfunc.load_model(model_uri_latest)
    
    # Perform a sample prediction
    sample_input = eval_df.head(1)
    logger.info(f"Performing sample prediction on: \n{sample_input['markdown'].iloc[0][:100]}...")
    
    prediction = loaded_model.predict(sample_input)
    logger.info(f"✅ Sample prediction successful. Output:\n{prediction.iloc[0][:100]}...")
except Exception as e:
    logger.error(f"Failed to validate registered model. Error: {e}")

In [ ]:
# %%time
logger.info("Starting model evaluation with mlflow.evaluate...")

# mlflow.evaluate requires a run to be active
with mlflow.start_run(run_id=run_id):
    results = mlflow.evaluate(
        model=model_uri,
        data=eval_df,
        targets="markdown",
        feature_names=["markdown"],
        evaluators="default",
        extra_metrics=[
            ari_grade_level(),
            exact_match(),
            rouge1(),
            rougeL(),
            semantic_similarity_metric,
            grammar_error_count_metric,
            grammar_error_rate_metric,
            grammar_improvement_metric,
            grammar_score_metric,
            readability_improvement_metric,
            llm_judge_metric_local
        ]
    )

    logger.info("✅ Evaluation complete.")
    logger.info("Evaluation Metrics:")
    for key, value in results.metrics.items():
        logger.info(f"  - {key}: {value:.4f}")


In [ ]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")